## Scraping Detik

In [107]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [108]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [109]:
def scrape_links(page_number):
    url = f"https://news.detik.com/indeks/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article', {"class": "list-content__item"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [110]:
jumlah_index = 50
threads_link = []
links = []
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 20 links from page 11
Scraped 20 links from page 2
Scraped 20 links from page 4
Scraped 20 links from page 12
Scraped 20 links from page 10
Scraped 20 links from page 9
Scraped 20 links from page 20
Scraped 20 links from page 19
Scraped 20 links from page 15
Scraped 20 links from page 18
Scraped 20 links from page 14
Scraped 20 links from page 7
Scraped 20 links from page 3
Scraped 20 links from page 13
Scraped 20 links from page 46
Scraped 20 links from page 35
Scraped 20 links from page 45
Scraped 20 links from page 34
Scraped 20 links from page 49
Scraped 20 links from page 42
Scraped 20 links from page 28
Scraped 20 links from page 48
Scraped 20 links from page 25
Scraped 20 links from page 38
Scraped 20 links from page 44
Scraped 20 links from page 5
Scraped 20 links from page 32
Scraped 20 links from page 37
Scraped 20 links from page 16
Scraped 20 links from page 8
Scraped 20 links from page 29
Scraped 20 links from page 17
Scraped 20 links from page 33
Scraped 20 links 

In [111]:
results = []
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        # title =[]
        # author = []
        # date = []
        # content = []
        # category =[]
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1', {"class": "detail__title"})
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "detail__author"})
        if author_elem:
            author_text = author_elem.get_text()
            author_text = author_text.split('-')[0].strip()
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "detail__date"})
        if date_elem:
            date_text = date_elem.text.strip()
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('div',{"class": "page__breadcrumb"})
        if category_elements:
            category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
            category_text= category_text.text.strip()
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('div', {"class": "detail__body"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [112]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://20.detik.com/detikupdate/20230925-230925073/momen-hangat-silaturahmi-ganjar-dengan-keluarga-besar-transmedia


In [113]:
df = pd.DataFrame(results)

df.head(10)

,title,author,category,date,content,link
0,PSI Angkat Giring Ganesha Jadi Dewan Pembina,Firda Cynthia Anggrainy,Pemilu,"Senin, 25 Sep 2023 19:39 WIB",Giring Ganesha diangkat pada jabatan baru seba...,https://news.detik.com/pemilu/d-6950217/psi-an...
1,Kaesang Pangarep Jadi Ketum PSI!,"Firda Cynthia Anggrainy, Adrial Akbar",Pemilu,"Senin, 25 Sep 2023 19:43 WIB","Putra bungsu Presiden Joko Widodo (Jokowi), Ka...",https://news.detik.com/pemilu/d-6950223/kaesan...
2,"Diantar Naik Motor, Anies Datangi Baintelkam P...",Adrial Akbar,Pemilu,"Senin, 25 Sep 2023 11:58 WIB",Bakal calon presiden (bacapres) Koalisi Peruba...,https://news.detik.com/pemilu/d-6949111/dianta...
3,KPK Puji Pabrik Pengelolaan Sampah Jadi Bahan ...,Jihaan Khoirunnisaa,Berita,"Senin, 25 Sep 2023 17:17 WIB",Komisi Pemberantasan Korupsi (KPK) mengapresia...,https://news.detik.com/berita/d-6949939/kpk-pu...
4,"Pemkot Depok Kucurkan Rp 2,7 M untuk Trotoar R...",Devi Puspitasari,Berita,"Senin, 25 Sep 2023 11:30 WIB",Dinas Pekerjaan Umum dan Penataan Ruang (DPUPR...,https://news.detik.com/berita/d-6949055/pemkot...
5,"Kesulitan Air, Warga Perumahan di Bekasi Ambil...",Kurniawan Fadil,Berita,"Senin, 25 Sep 2023 19:43 WIB","Warga di Perumahan Bali Indah, Setiamulya, Tar...",https://news.detik.com/berita/d-6950224/kesuli...
6,5 Fakta Demo Tutup Suramadu Protes Truk Garam ...,Tim detikJatim,Berita,"Senin, 25 Sep 2023 17:30 WIB",Demo akan dilakukan sejumlah massa dari organi...,https://news.detik.com/berita/d-6949980/5-fakt...
7,Mayat Pria Terbakar Ditemukan di Pos Area Lanu...,Wildan Noviansah,Berita,"Senin, 25 Sep 2023 19:02 WIB",Seorang pria ditemukan tewas dalam kondisi ter...,https://news.detik.com/berita/d-6950156/mayat-...
8,Pemotor Lawan Arah Tewaskan Pesepeda di Jakut ...,Wildan Noviansah,Berita,"Senin, 25 Sep 2023 18:55 WIB","Ulah pemotor lawan arah di Cilincing, Jakarta ...",https://news.detik.com/berita/d-6950145/pemoto...
9,Penampilan Siskaeee Jalani Pemeriksaan Kasus '...,Author not found,Category not found,"14,459 Views | Senin, 25 Sep 2023 10:41 WIB",Siskaeee hadiri undangan pemeriksaan penyidik ...,https://20.detik.com/detikupdate/20230925-2309...


Save Data

In [114]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/detik_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/detik_2023-09-25_20-47-39.xlsx


In [115]:
# links=[]
# # jumlah Page yang diambil 
# jumlah_index = 20

# for index in range(jumlah_index):
#     url = f"https://news.detik.com/indeks/{index+1}"  
#     # print(url)
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     articles = soup.find_all('article', {"class": "list-content__item"})
    
#     for article in articles:
#         link = article.find('a')['href']
#         links.append(link)
    
# print("Total Links",len(links))

In [116]:
# title = []
# author = []
# date = []
# content = []
# category =[]

# for link in links:
#     url = link
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Judul Berita
#     title_elem = soup.find('h1', {"class": "detail__title"})
#     if title_elem:
#         title_text = title_elem.text.strip()
#     else:
#         title_text = "Title not found"
#     title.append(title_text)
    
#     # Author berita
#     author_elem = soup.find('div', {"class": "detail__author"})
#     if author_elem:
#         author_text = author_elem.get_text()
#         author_text = author_text.split('-')[0].strip()
#     else:
#         author_text = "Author not found"
#     author.append(author_text)
    
#     # tanggal berita
#     date_elem = soup.find('div', {"class": "detail__date"})
#     if date_elem:
#         date_text = date_elem.text.strip()
#     else:
#         date_text = "Date not found"
#     date.append(date_text)
    
#     # Category berita
#     category_elements = soup.find('div',{"class": "page__breadcrumb"})
#     if category_elements:
#         category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
#         category_text= category_text.text.strip()
#     else:
#         category_text = "Category not found"
#     category.append(category_text)
   
    
#     # Content Berita
#     body_elem = soup.find('div', {"class": "detail__body"})
    
#     if body_elem:
#         content_elem = body_elem.find_all('p')
#         content_text = ""
#         for p in content_elem:
#             content_text += p.text.strip() + "\n"
        
#         if content_text.strip():
#             content.append(content_text)
#         else:
#             content.append("Content not found")
#     else:
#         content.append("Content not found")
    
    

In [117]:
# data = {'title': title,
#         'author' : author,
#         'category':category,
#         'date': date,
#         'content' : content,
#         'link' : links
#         }

# df = pd.DataFrame(data)

# df.head(10)

In [118]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/detik_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/detik_2023-09-25_20-47-40.xlsx


In [119]:
# # Latihan Pengecek dan find element
# url = f"https://news.detik.com/detiknews/d-6948823/siskaeee-penuhi-panggilan-polisi-soal-film-porno-ini-penampilannya"  
# # print(url)
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")


# Category
# category_elements = soup.find('div',{"class": "page__breadcrumb"})
# print(category_elements)
# category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
# print(category_text.text.strip())

# ambil author detik
# title_elem = soup.find('div', {"class": "detail__author"})
# author_text = title_elem.get_text()
# author_name = author_text.split('-')[0].strip()
# print(author_name)

In [120]:
# def scrape_url(url):
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#     else:
#         print(f"Failed to retrieve data from {url}")

In [121]:
# urls = ['https://kompas.com', 'https://kompas.com', 'https://kompas.com']
# threads = []

In [122]:
# for url in urls:
#     thread = threading.Thread(target=scrape_url, args=(url,))
#     thread.start()
#     threads.append(thread)

# print(threads)

In [123]:
# for thread in threads:
#     thread.join()
    
# print("All threads finished")